# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-V3',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/PowerInfer/SmallThinker-3B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/Qwen/QVQ-72B-preview',
 '/spaces',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company social media', 'url': 'https://twitter.com/huggingface'},
  {'type': 'company social media',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'forums', 'url': 'https://discuss.huggingface.co'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepse

In [24]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3\nUpdated\n4 days ago\n•\n54.8k\n•\n1.03k\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n4 days ago\n•\n7.53k\n•\n1.11k\nPowerInfer/SmallThinker-3B-Preview\nUpdated\n1 day ago\n•\n1.44k\n•\n178\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.17M\n•\n7.69k\nhexgrad/Kokoro-82M\nUpdated\nabout 20 hours ago\n•\n449\n•\n165\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n2.46k\n🏢\nTRELLIS\nScalable and Versati

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face

### The AI Community Building the Future

At Hugging Face, we are at the forefront of the artificial intelligence revolution, serving as a collaborative platform for the machine learning community. Our goal is to empower developers, researchers, and organizations to create, discover, and share advanced models, datasets, and applications in a supportive environment. 

---

## What We Offer

- **Models**: Access over **400,000** machine learning models catering to a variety of tasks.
- **Datasets**: Explore a vast library of **100,000+ datasets** for diverse machine learning applications.
- **Spaces**: Innovate and run applications with our easy-to-use Spaces feature, which hosts more than **150,000 applications**.
- **Enterprise Solutions**: Tailored for organizations with specialized needs, our enterprise-grade solutions offer security, support, and scalability.

---

## Our Community

Join more than **50,000 organizations** worldwide, including major players like **Meta, Amazon Web Services, Google**, and **Microsoft**. 
- Experience collaborative learning and creation with fellow innovators. 
- Build your portfolio and enhance your machine learning profile. 

---

### Company Culture

At Hugging Face, we are deeply committed to creating an open-source environment that fosters collaboration and shared success. We believe in the power of community and the collective knowledge that shapes the future of machine learning. 

- **Open-Source Commitment**: We're building the foundation of machine learning tooling in conjunction with the community.
- **Diversity of Thought**: Our culture thrives on diverse perspectives and inclusive practices to drive innovative solutions.

---

## Career Opportunities

We are constantly looking for passionate individuals to join our team and contribute to our mission of democratizing AI. Current openings include a variety of positions across departments such as engineering, research, and community support.

- **Professional Growth**: Engage in a culture of continuous learning and development.
- **Collaborative Environment**: Work alongside some of the brightest minds in AI.

Check our [Careers Page](#) for the latest openings and join our journey today!

---

## Get Started Today!

- **For Developers**: Sign up for free to start building and sharing your machine learning models.
- **For Organizations**: Explore our enterprise solutions tailored to meet your needs.

Visit us at [Hugging Face](https://huggingface.co) and become part of a revolutionary community! 

--- 

### Connect with Us
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **GitHub**: [Hugging Face GitHub](https://github.com/huggingface)
- **LinkedIn**: [Hugging Face LinkedIn](https://linkedin.com/company/hugging-face)
- **Discord**: Join our community on Discord for real-time collaboration!

Together, let’s build the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

---

## Who We Are
**Hugging Face** is at the forefront of the AI revolution. We are a community-driven platform dedicated to building the future of machine learning (ML). Our mission is to democratize access to state-of-the-art ML tools and foster collaboration in AI development.

---

## Our Offerings
### Collaboration Platform
Hugging Face facilitates collaboration on:
- **Models**: Over **400,000+** models available for diverse applications including natural language processing, computer vision, and audio.
- **Datasets**: Access to **100,000+** datasets curated for various machine learning tasks.
- **Spaces**: Explore a variety of applications and demo projects that showcase cutting-edge AI capabilities.

### Open Source Community
We contribute to the foundation of ML tooling through our extensive open source library, which includes:
- **Transformers**: The go-to library for implementing state-of-the-art ML architectures.
- **Diffusers**: Advanced models for image and audio generation.
- **Tokenizers**, **PEFT**, and more, ensuring both research and production-grade tools are at your fingertips.

### Enterprise Solutions
Hugging Face offers tailored services for over **50,000 organizations**, providing them with:
- **Enterprise-grade security**
- **Dedicated support**
- Flexible pricing starting at **$20/user/month**.

---

## Company Culture
At Hugging Face, we believe in fostering an inclusive and collaborative environment. Our team comprises over **223 dedicated members**, where every individual has a voice and contributes to our shared mission. We are committed to transparency, continuous learning, and community engagement—ensuring everyone feels empowered to innovate and share their work.

---

## Join Us
### Careers at Hugging Face
We are always looking for passionate individuals committed to pushing the boundaries of AI and ML. Check out our careers page to find opportunities where you can make an impact in the AI community.

**Explore Opportunities:** [Careers](https://huggingface.co/jobs)

---

## Our Customer Base
More than **50,000 organizations**, including giants like:
- **Meta**
- **Google**
- **Amazon Web Services**
- **Microsoft**
- **Grammarly**

We are proud to serve a diverse clientele ranging from startups to established enterprises.

---

## Get Involved
Dive into the world of machine learning with us! Whether you are a learner, a professional, or a business looking to leverage AI—Hugging Face is your platform of choice.

**Join the Community:** [Sign Up](https://huggingface.co/signup)

---

*Transforming the future of AI, one commit at a time.*

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to the Hugging Face Brochure!

## 🤗 Hugging Face: The AI Community Building the Future!

### Where Machine Learning Meets a Hug!

At Hugging Face, we believe that AI should be as friendly as your favorite teddy bear. We’re the platform where the machine learning community collaborates on models, datasets, and applications all while possibly engaging in the world’s largest group hug (virtually, of course)!

---

### 🌟 Trending Models of the Week!

Join us in celebrating our model stars! Meet the latest creations that have everyone talking (and possibly hugging):

- **DeepSeek-V3**: Updated 4 days ago and already over 54.8k fans. It's the *rockstar* of ML.
- **SmallThinker-3B-Preview**: Too smart for its own good with 1.44k in popularity!
- And let's not forget **FLUX.1-dev**, boasting a jaw-dropping **1.17M** followers! Can you say *celebrity*? 

(Warning: If you’re feeling inadequate, just know that they have many, many more parameters than you.)

---

### 💼 Careers at Hugging Face!

Are you looking for a *hug* that pays? We're on a mission to democratize machine learning, one friendly face at a time! Join our team of AI enthusiasts who enjoy:

- 🧑‍💻 Collaborating on innovative projects!
- 🥳 Enjoying an incredible company culture (yes, we have snacks!).
- 🦸‍♂️ Learning opportunities that will build your AI superpowers.

**Open Positions**: We’re hiring more than just data wranglers—think AI superheroes ready to take on the challenges of ML! 

---

### 📊 Who Are Our Customers? 

Over **50,000 organizations** trust Hugging Face to bring AI into their operations. From tech giants like **Google** and **Microsoft** to cool startups you’ve probably never heard of, everyone is ready to ride the Hugging Face wave!

---

### 💡 What We Do!

- Hugging Face is not just a cute name; we’re the backbone of the machine learning community with over **400k models** and **150k applications** to explore.
- Our **open-source** software is so user-friendly you might want to give it a hug (it’s socially acceptable here).
- With tools for text, image, video, audio, and even 3D, we ensure you can get your creativity on!

---

### 🎉 Join the Hug!

If you're ready to step into a community where collaboration is key and friendliness reigns supreme, **sign up** and bring your best ideas. Because who knew AI could feel like a bear hug?

---

**For more info, follow the links or just give us a friendly nudge. Till then, keep it human, keep it loving!** 

 *Disclaimer: No actual hugging occurs at our headquarters. But you might experience a comforting sense of belonging.* 

--- 

**Connect with us!**  
[Twitter](https://twitter.com/huggingface) | [GitHub](https://github.com/huggingface) | [LinkedIn](https://linkedin.com/company/hugging-face)

**Stop by and say hi! We promise a warm welcome! 🤗**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>